In [1]:
import pandas as pd
import os
import csv
import json

In [2]:
os.listdir('./csv')

['.DS_Store',
 'dishwashers.csv',
 'washing machines.csv',
 'Fridges and Freezers.csv',
 'computer monitors.csv',
 'dryer.csv',
 'tv.csv']

In [3]:
all_dfs = {file.split('.')[0].strip(): pd.read_csv(os.path.join('./csv', file)).fillna('NaN')
           for file in os.listdir('./csv') if file.endswith('.csv')}

In [4]:
all_dfs.keys()

dict_keys(['dishwashers', 'washing machines', 'Fridges and Freezers', 'computer monitors', 'dryer', 'tv'])

In [5]:
# for auto wrangle all files
all_cols = set()

for each in all_dfs.values():
    print(len(each.columns))
    all_cols = all_cols.union(each.columns)
    
print(len(all_cols))

7
7
7
7
7
7
7


In [6]:
for df in all_dfs.values():
    #current_json = dict()
    #current_json[df['Appliance Type'].unique()] = dict()
    print(df['Appliance Type'].unique())

['Dishwashers']
['Washing Machines']
['Fridges and Freezers']
['Computer Monitor']
['Dryers']
['Televisions']


In [7]:
for f_n, df in all_dfs.items():
    print(f_n)
    print(df[df.duplicated(subset=['Brand Name', 'Model Number'])].shape)

dishwashers
(0, 7)
washing machines
(0, 7)
Fridges and Freezers
(0, 7)
computer monitors
(13, 7)
dryer
(0, 7)
tv
(0, 7)


In [8]:
all_dfs['computer monitors'].drop_duplicates(subset=[each for each in all_dfs['computer monitors'].columns 
                                                     if each != 'Unnamed: 0'], 
                                             inplace=True)


In [9]:
all_jsons = {}

for f_n, df in all_dfs.items():
    
    current_json = dict()
    current_json_content = dict()
    
    for brand_tmp in df['Brand Name'].unique():
        current_json_content[brand_tmp] = dict()
        
        for model_tmp in df.loc[df['Brand Name'] == brand_tmp, 'Model Number']:
            current_json_content[brand_tmp][model_tmp] = \
            dict(zip(['Manufacturing Countries', 'Comparative Energy Consumption', 'Star Rating'],
            df.loc[((df['Brand Name'] == brand_tmp) & (df['Model Number'] == model_tmp)),
                   ['Manufacturing Countries', 'Comparative Energy Consumption', 'Star Rating']].astype(str).values[0]
            ))
    current_json[df['Appliance Type'].unique()[0]] = current_json_content
    all_jsons[f_n] = current_json

In [10]:
for f_n, f_j in all_jsons.items():
    file_name = './json/{}.json'.format(f_n)
#     print(file_name)
    with open(file_name, mode='w', encoding='utf-8') as f:
        f.write(json.dumps(f_j))